In [7]:
import pandas as pd


conversation_parts = pd.read_csv("/content/CONVERSATION_PARTS.csv")
conversations = pd.read_csv("/content/CONVERSATIONS.csv")

# Afficher les premières lignes
print(conversation_parts.head())
print(conversations.head())

<ipython-input-7-723a50363b00>:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  conversation_parts = pd.read_csv("/content/CONVERSATION_PARTS.csv")


  ASSIGNED_TO ATTACHMENTS                                             AUTHOR  \
0         NaN          []  {\n  "id": "5ebba13923665421f7051ae0",\n  "typ...   
1         NaN          []        {\n  "id": "5217337",\n  "type": "admin"\n}   
2         NaN          []           {\n  "id": "775489",\n  "type": "bot"\n}   
3         NaN          []        {\n  "id": "5217337",\n  "type": "admin"\n}   
4         NaN          []        {\n  "id": "5217337",\n  "type": "admin"\n}   

     CONVERSATION_CREATED_AT  CONVERSATION_ID    CONVERSATION_UPDATED_AT  \
0  2022-01-31 14:02:02.000 Z   53815801349742  2022-03-09 10:10:18.000 Z   
1  2022-01-31 14:02:02.000 Z   53815801349742  2022-03-09 10:10:18.000 Z   
2  2022-01-31 14:02:02.000 Z   53815801349742  2022-03-09 10:10:18.000 Z   
3  2022-01-31 14:02:02.000 Z   53815801349742  2022-03-09 10:10:18.000 Z   
4  2022-01-31 14:02:02.000 Z   53815801349742  2022-03-09 10:10:18.000 Z   

                  CREATED_AT           ID                NOTIF

In [17]:
import pandas as pd
import json

def safe_json_loads(x):
    if pd.isna(x) or x.strip() == '':
        return {}
    try:
        return json.loads(x.replace('""', '"'))
    except Exception:
        return {}

# 1. Charger le CSV
df = pd.read_csv('/content/CONVERSATIONS.csv')

# 2. Parser la colonne CONVERSATION_RATING
conversation_rating = df['CONVERSATION_RATING'].apply(safe_json_loads)

df['rating'] = conversation_rating.apply(lambda x: x.get('rating'))
df['remark'] = conversation_rating.apply(lambda x: x.get('remark'))
df['teammate_id'] = conversation_rating.apply(lambda x: x.get('teammate', {}).get('id'))
df['teammate_type'] = conversation_rating.apply(lambda x: x.get('teammate', {}).get('type'))

# 3. Parser la colonne ASSIGNEE
assignee = df['ASSIGNEE'].apply(safe_json_loads)
df['assignee_id'] = assignee.apply(lambda x: x.get('id'))
df['assignee_type'] = assignee.apply(lambda x: x.get('type'))

# 4. Parser la colonne TAGS (qui est une liste)
tags = df['TAGS'].apply(safe_json_loads)

def extract_tag_names(tag_list):
    if isinstance(tag_list, list):
        return [tag.get('name') for tag in tag_list if 'name' in tag]
    return []

df['tags_list'] = tags.apply(extract_tag_names)
df['tags_concat'] = df['tags_list'].apply(lambda tags: ", ".join(tags) if tags else None)

# 5. Drop les anciennes colonnes JSON si besoin
df_cleaned = df.drop(columns=['CONVERSATION_RATING', 'ASSIGNEE', 'TAGS'])

# 6. Sauvegarder dans un nouveau CSV
df_cleaned.to_csv('conversations_cleaned.csv', index=False)

print(" Nouveau fichier conversations_cleaned.csv généré.")


 Nouveau fichier conversations_cleaned.csv généré.


In [2]:
import pandas as pd
import json

# Charger le fichier CSV
conversation_parts = pd.read_csv('/content/CONVERSATION_PARTS.csv')

# Fonction pour parser la colonne 'AUTHOR'
def parse_author(author):
    if pd.isna(author) or author.strip() == '':
        return pd.Series({'author_id': None, 'author_type': None})
    try:
        author_json = json.loads(author)
        return pd.Series({
            'author_id': author_json.get('id'),
            'author_type': author_json.get('type')
        })
    except json.JSONDecodeError:
        print(f"Erreur JSON pour l'author : {author}")
        return pd.Series({'author_id': None, 'author_type': None})

# Appliquer la fonction sur la colonne 'AUTHOR'
author_info = conversation_parts['AUTHOR'].apply(parse_author)

# Ajouter les nouvelles colonnes au DataFrame
conversation_parts = pd.concat([conversation_parts, author_info], axis=1)

# (Optionnel) Supprimer la colonne AUTHOR si tu n'en as plus besoin
conversation_parts = conversation_parts.drop(columns=['AUTHOR','ATTACHMENTS','ASSIGNED_TO'])

# Sauvegarder dans un nouveau CSV
conversation_parts.to_csv('conversation_part_clean.csv', index=False)

print("Fichier conversation_part_clean.csv généré avec succès.")


<ipython-input-2-5cfe0e635385>:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  conversation_parts = pd.read_csv('/content/CONVERSATION_PARTS.csv')


Fichier conversation_part_clean.csv généré avec succès.
